In [1]:
from Bio import SeqIO
import re
import csv
import pandas as pd

In [ ]:
existing_table = 'metadata_pools.xlsx'  
df_existing = pd.read_excel(existing_table)  

gb_file = '1485_genbank.gb'
genbank_records = list(SeqIO.parse(gb_file, 'gb'))

gb_dict = {}
for record in genbank_records:
    accession = record.id.split('.')[0] 
    gb_dict[accession] = record

reference_titles = []
isolation_sources = []

for accession in df_existing['Genbank accession']:
    acc_str = str(accession).strip()
    
    if acc_str in gb_dict:
        record = gb_dict[acc_str]
        
        reference_title = "No reference title found"
        if record.annotations.get('references'):
            reference_title = record.annotations['references'][0].title
        
        isolation_source = "Not found"
        for feature in record.features:
            if feature.type == "source" and 'isolation_source' in feature.qualifiers:
                isolation_source = feature.qualifiers['isolation_source'][0]
                break
        
        reference_titles.append(reference_title)
        isolation_sources.append(isolation_source)
    else:
        print(f"Warning: Accession {acc_str} not found in GenBank file")
        reference_titles.append("Not found")
        isolation_sources.append("Not found")

df_existing['Reference Title'] = reference_titles
df_existing['Isolation source'] = isolation_sources

output_file = 'updated_table_with_reference_titles.xlsx'
df_existing.to_excel(output_file, index=False)